# Vector Space Model

Text preprocessing piplines:
Text denoising -> text normalization -> text standarization

In this part, we will conduct text standarization using vector space model.

1. Stop words, ngrams, the whole pipeline of text preprocessing.
2. Bag-of-word representation
3. Term weighting: 
    - Term frequency
    - Inverse document frequency
    - TF-IDF

## 1. Stop words, ngrams, the whole pipeline of text preprocessing.

Stopwords's intuition: Not all words are informative

- Remove such words to reduce vocabulary size
- Stopwords do not have universal definition
- Risk: we may break the original meaning and structure of text when we remove some words.

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
print(stopwords)

{'doesn', 'me', 'd', 'but', 'into', 'have', 'if', 'no', 'this', 'while', 'mightn', 'needn', 'they', 'most', 'up', 'which', 'during', 'there', 'by', 'she', 'any', 'that', 'had', "wasn't", 'those', 'than', 'of', 'under', 'don', 'from', "couldn't", 'herself', 'you', 'won', 'ours', 'yourself', 'above', "weren't", 'so', 'with', 'hasn', 'few', 'should', 'ma', 'further', 'been', 'wouldn', 'these', "it's", 'couldn', 'didn', "needn't", 'its', 'whom', 'below', 'our', 'myself', 'only', "that'll", 'hers', 'or', 'wasn', 'as', 'themselves', "shan't", 'itself', "you'd", 'be', 'do', 'against', "didn't", 'about', 'then', 't', 'between', 'nor', 'am', 'being', 'out', 'can', 'at', 'hadn', 'll', 'ain', "mightn't", 'off', 'until', 'to', 'both', 'weren', "she's", "won't", 'what', "doesn't", 'through', "mustn't", 'their', 'before', 'all', 'after', 'y', "don't", 'the', 'has', 'and', 'a', 'same', 'doing', "you'll", 'other', 'm', 'where', 'not', 'mustn', 'such', 'i', "wouldn't", 'when', "shouldn't", "hasn't", 'h

In [3]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def tokenize(text):
    """
    :param text: a doc with multiple sentences, type: str
    return a word list, type: list
    e.g.
    Input: 'Text mining is to identify useful information.'
    Output: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    """
    return nltk.word_tokenize(text)

def stem(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of stemmed words, type: list
    e.g.
    Input: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    Output: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    """

    return [ps.stem(token) for token in tokens]

def filter_stopwords(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of filtered tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    Output: ['text', 'mine', 'identifi', 'use', 'inform', '.']
    """

    return [token for token in tokens if token not in stopwords and not token.isnumeric()]

In [4]:
tokens = tokenize("Text mining is to identify useful information.")
print(tokens)

tokens = stem(tokens)
print(tokens)

print(filter_stopwords(tokens))

['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
['text', 'mine', 'identifi', 'use', 'inform', '.']


A single word is sometimes weakly expressive so that we can try n-gram for better representation

In [5]:
def n_gram(tokens, n=1):
    """
    :param tokens: a list of tokens, type: list
    :param n: the corresponding n-gram, type: int
    return a list of n-gram tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.'], 2
    Output: ['text mine', 'mine is', 'is to', 'to identifi', 'identifi use', 'use inform', 'inform .']
    """
    if n == 1:
        return tokens
    else:
        results = list()
        for i in range(len(tokens)-n+1):
            # tokens[i:i+n] will return a sublist from i th to i+n th (i+n th is not included)
            results.append(" ".join(tokens[i:i+n]))
        return results

In [6]:
bi_gram = n_gram(tokens, 2)
print(bi_gram)

['text mine', 'mine is', 'is to', 'to identifi', 'identifi use', 'use inform', 'inform .']


In [7]:
tri_gram = n_gram(tokens, 3)
print(tri_gram)

['text mine is', 'mine is to', 'is to identifi', 'to identifi use', 'identifi use inform', 'use inform .']


## 2. Bag-of-words representation

Converting the words to numerical features.

## One-hot vector
A one-hot is a group of bits among which the legal combinations of values are only those with a single high (1) bit and all the others low (0).

For an example sentence ```text mining is good```, we map all the words into indexes: map ```text``` to 0, ```mining``` to 1, ```is``` to 2, and ```good``` to 3.

Then the one hot vector for ```text``` is ```[1, 0, 0, 0]```. For ```mining``` it is ```[0, 1, 0, 0]```, and so on.

## Bag of word (BOW)
The BOW vector of a sentence is the sum of all the one-hot vectors.

For ```text mining is good```, the BOW representation is ```[1, 1, 1, 1]```.

For ```text mining good```, the BOW representation is ```[1, 1, 0, 1]```.

For ```text mining is good mining```, the BOW representation is ```[1, 2, 1, 1]```.

In [8]:
import numpy as np

def get_vocabulary_mapping(tokens):
    """
    :param tokens: a list of word tokens type: list
    
    return vocab_dict: a dict from words to indices, type: dict
    """
    vocab_dict = {}
    for token in tokens:
        if not token in vocab_dict:
            vocab_dict[token] = len(vocab_dict)
    return vocab_dict

def get_bow_vector(tokens, vocab_dict):
    """
    :param tokens: a list of tokenized words, type: list
    :param vocab_dict: a dict from words to indices, type: dict
    
    return a feature vector,
    """
    # initialize the vector as all zeros
    vector = np.zeros(len(vocab_dict), dtype=np.float)
    for f in tokens:
        # get the feature index, return -1 if the feature is not existed
        f_idx = vocab_dict.get(f, -1)
        if f_idx != -1:
            # set the corresponding element as 1
            vector[f_idx] = vector[f_idx] + 1
    return vector

In [9]:
vocab = get_vocabulary_mapping(["text", "mining", "is", "good"])

In [10]:
print(vocab)

{'text': 0, 'mining': 1, 'is': 2, 'good': 3}


In [11]:
tokens_1 = tokenize("text mining is good mining.")
tokens_2 = tokenize("text is good")
tokens_3 = tokenize("good text is good")
print(get_bow_vector(tokens_1, vocab))
print(get_bow_vector(tokens_2, vocab))
print(get_bow_vector(tokens_3, vocab))

[1. 2. 1. 1.]
[1. 0. 1. 1.]
[1. 0. 1. 2.]


In [12]:
def calculate_bow(corpus):
    """
        :param corpus: a list of strings (type: list)
    """
    
    
    def vectorize(sentence, vocab):
        """
           :param sentence: a string (type:str)
           :param vocab: vocabulary (type:list) 
           
           count function of a list is to count the time of occurance in a list
           :return BOW vector
        """
        return [sentence.split().count(i) for i in vocab]

    vectorized_corpus = []
    vocab = sorted(set([token for doc in corpus for token in doc.lower().split()]))
    for sent in corpus:
        vectorized_corpus.append((sent, vectorize(sent, vocab)))
    return vectorized_corpus, vocab

In [13]:
all_sents = ["text mining is good mining",
"text is good",
"good text is good"]
corpus_bow, vocab = calculate_bow(all_sents)
print(corpus_bow)
print(vocab)

[('text mining is good mining', [1, 1, 2, 1]), ('text is good', [1, 1, 0, 1]), ('good text is good', [2, 1, 0, 1])]
['good', 'is', 'mining', 'text']


## Calculate similarity

In [14]:
from math import sqrt
def cosine_sim(u,v):
    return np.dot(u,v) / (sqrt(np.dot(u,u)) * sqrt(np.dot(v,v)))

def print_similarity(corpus):
    """
    Print pairwise similarities
    """
    for sentx in corpus:
        for senty in corpus:
            print("{:.4f}".format(cosine_sim(sentx[1], senty[1])), end=" ")
        print()
    print()

In [15]:
print_similarity(corpus_bow)
# ["text mining is good mining",
# "text is good",
# "good text is good"]

1.0000 0.6547 0.6172 
0.6547 1.0000 0.9428 
0.6172 0.9428 1.0000 



## 3. Term weighting
Term frequency: number of times term ***t*** appearing in document ***d***

Inverse document frequency: total number of docs in collection over number of docs containing term t

TF-IDF

In [16]:
def term_freq(freq_dict, term):
    """
        :param freq_dict (type:dict): a dict variable whose key is the word and value is the frequency.
            e.g., dict(["text":1, "mining":1, "is":1])
        :param term (type:str): the candidate word to calculate TF
        
        :returns, the TF score of a certain word
    """
    try:
        return freq_dict[term] / float(sum(freq_dict.values()))
    except ZeroDivisionError:
        return 0

def inverse_doc_freq(freq_dict_list, term):
    """
        :param freq_dict_list (type: list): a list of freq_dict 
        :param term (type:str): the candidate word to calculate TF
        
        :returns, the IDF of a certain word
    """
    try:
        unique_docs = sum([1 for i,_ in enumerate(freq_dict_list) if freq_dict_list[i][term] > 0])
        return float(len(freq_dict_list)) / unique_docs
    except ZeroDivisionError:
        return 0

In [17]:
# sentence: good text is good
current_bow = corpus_bow[2]
print("bow representation:", current_bow)
doc_vec_dict = {k:v for k,v in zip(vocab, current_bow[1])}
print("frequency dict:", doc_vec_dict)

bow representation: ('good text is good', [2, 1, 0, 1])
frequency dict: {'good': 2, 'is': 1, 'mining': 0, 'text': 1}


In [18]:
print("TF: good", term_freq(doc_vec_dict, "good"))
print("TF: text", term_freq(doc_vec_dict, "text"))
print("TF: is", term_freq(doc_vec_dict, "is"))

TF: good 0.5
TF: text 0.25
TF: is 0.25


In [19]:
freq_dict_list = [{k:v for k,v in zip(vocab, vecs[1])} for vecs in corpus_bow]
print(freq_dict_list)

[{'good': 1, 'is': 1, 'mining': 2, 'text': 1}, {'good': 1, 'is': 1, 'mining': 0, 'text': 1}, {'good': 2, 'is': 1, 'mining': 0, 'text': 1}]


In [20]:
# all_sents = [
# "text mining is good mining",
# "text is good",
# "good text is good"]
print("IDF: good", inverse_doc_freq(freq_dict_list, "good"))
print("IDF: text", inverse_doc_freq(freq_dict_list, "text"))
print("IDF: is", inverse_doc_freq(freq_dict_list, "is"))
print("IDF: mining", inverse_doc_freq(freq_dict_list, "mining"))

IDF: good 1.0
IDF: text 1.0
IDF: is 1.0
IDF: mining 3.0


In [21]:
def calculate_tfidf(corpus_bow, vocab):
    """
        :params corpus_bow(type: list): the BOW representation of the corpus
        :params vocab (type:list): the list of vocab
        
        return the tf idf representation of the corpus
    """
    word2id = dict(zip(vocab, range(len(vocab))))

    freq_dict_list = [{k:v for k,v in zip(vocab, i[1])} for i in corpus_bow]
    tfidf_mat  =  np.zeros((len(freq_dict_list), len(vocab)), dtype=float)
    for doc_id, doc in enumerate(freq_dict_list):
        for term in doc:
            term_id = word2id[term]
            tf = term_freq(freq_dict_list[doc_id],term)
            idf = inverse_doc_freq(freq_dict_list, term)
            tfidf_mat[doc_id][term_id] = tf*idf

    all_sents = [doc[0] for doc in corpus_bow]
    corpus_tfidf = list(zip(all_sents, tfidf_mat))
    return corpus_tfidf

In [22]:
corpus_bow_tfidf = calculate_tfidf(corpus_bow, vocab)

In [23]:
list(zip(corpus_bow, corpus_bow_tfidf))

[(('text mining is good mining', [1, 1, 2, 1]),
  ('text mining is good mining', array([0.2, 0.2, 1.2, 0.2]))),
 (('text is good', [1, 1, 0, 1]),
  ('text is good', array([0.33333333, 0.33333333, 0.        , 0.33333333]))),
 (('good text is good', [2, 1, 0, 1]),
  ('good text is good', array([0.5 , 0.25, 0.  , 0.25])))]

In [24]:
vocab

['good', 'is', 'mining', 'text']